# Test Notebook for Building Pipeline Using Module 'sleep_infinity'
The module 'sleep_infinity' was generated through command: 'az ml module init -n sleep_infinity'

In [2]:
from azureml.core import Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.pipeline.wrapper import Module, Pipeline, dsl
import sys
from pathlib import Path
# The following line adds source directory to path. 
from sleep_infinity import sleep_infinity

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception cannot import name '_update_args_and_inputs' from 'azureml.core.script_run_config' (C:\Users\jietong\AppData\Roaming\Python\Python38\site-packages\azureml\core\script_run_config.py).


## Prerequisites: Configure workspace & compute

 - Update .azureml/config.json to make sure it has information on your workspace, subscription id, etc.
 - Change the following aml_compute_target's value to reference an existing compute target

In [3]:
workspace = Workspace.from_config(path = '.azureml/config.json')
compute_target = "itp-eastus-p40-2"
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id, compute_target, sep='\n')

itp-pilot
itp-pilot-ResGrp
westeurope
4aaa645c-5ae2-4ae9-a17a-84b9023bc56a
itp-eastus-p40-2


# Load module(s)
Load a module function through one of the following:
- Module.from_func: when you have a python function decorated with @dsl.module
- Module.from_yaml: when loading from existing module yaml definition
- Module.load: when you have already registered the module into Azure ML workspace

In [4]:
module_func = Module.from_func(workspace, sleep_infinity)
# help(module_func)

# Create pipeline
This is done by calling module function(s) with the parameters/inputs/outputs the module supports

In [5]:
@dsl.pipeline(name='sleep_infinity_pipeline', description='A sample pipeline uses sleep_infinity for interactive job in ITP', default_compute_target=compute_target)
def sleep_pipeline() -> Pipeline:
    sleep_infinity = module_func()
    sleep_infinity.k8srunsettings.priorityconfiguration.is_preemptible=False
    sleep_infinity.k8srunsettings.resourceconfiguration.gpu_count = 1
    sleep_infinity.k8srunsettings.interactiveconfiguration.is_ssh_enabled = True

    return {}

test_pipeline = sleep_pipeline()

In [18]:
run = test_pipeline.submit(
    experiment_name='InteractiveSessions'
)
run

Submitted PipelineRun 485d991b-a4d9-41af-8e97-da3fcfd983f7
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/sleep_infinity_experiment/runs/485d991b-a4d9-41af-8e97-da3fcfd983f7?wsid=/subscriptions/4aaa645c-5ae2-4ae9-a17a-84b9023bc56a/resourcegroups/groot_workspaces/workspaces/GrootEastUs


Experiment,Id,Type,Status,Details Page,Docs Page
sleep_infinity_experiment,485d991b-a4d9-41af-8e97-da3fcfd983f7,azureml.PipelineRun,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


# We can also run the interactive job without involving pipeline

In [7]:
sleep_infinity = module_func()

sleep_infinity.k8srunsettings.priorityconfiguration.is_preemptible=False
sleep_infinity.k8srunsettings.resourceconfiguration.gpu_count = 1
sleep_infinity.k8srunsettings.interactiveconfiguration.is_ssh_enabled = True
sleep_infinity.runsettings.configure(target=compute_target)

sleep_infinity.submit(experiment_name='InteractiveSessions')

[Warning] script_dir is None, create tempdir: C:\Users\jietong\AppData\Local\Temp\003f0329-db5e-452b-8a9d-459573ccd7fc
[Warning] Can't find sleep_infinity.py from C:\Users\jietong\AppData\Local\Temp\003f0329-db5e-452b-8a9d-459573ccd7fc, will download from remote
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/InteractiveSessions/runs/InteractiveSessions_1598430935_497a4c52?wsid=/subscriptions/4aaa645c-5ae2-4ae9-a17a-84b9023bc56a/resourcegroups/itp-pilot-ResGrp/workspaces/itp-pilot


Experiment,Id,Type,Status,Details Page,Docs Page
InteractiveSessions,InteractiveSessions_1598430935_497a4c52,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation
